In [2]:
import tensorflow as tf
import numpy as np

### Generate data points

In [3]:
def sample_points(k):
    
    num_points = 100
    
    #amplitude
    amplitude = np.random.uniform(low=0.1, high=5.0)
    
    #phase
    phase = np.random.uniform(low=0, high=np.pi)

    x = np.linspace(-5, 5, num_points)

    #y = a*sin(x+b)
    y = amplitude * np.sin(x + phase)
    
    #sample k data points
    sample = np.random.choice(np.arange(num_points), size=k)
    
    return (x[sample], y[sample])

In [5]:
x, y = sample_points(5)
print(x);
print(y);

[ 1.36363636  4.5959596   1.26262626 -4.09090909  4.49494949]
[-0.78438094  0.90049241 -0.64756805 -1.50086945  0.77080053]


### Two Layered Neural Network

In [7]:
#tf.reset_default_graph()
tf.compat.v1.reset_default_graph()

In [8]:
num_hidden = 64
num_classes = 1
num_feature = 1

In [14]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [15]:
X = tf.placeholder(tf.float32, shape=[None, num_feature])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

In [16]:
w1 = tf.Variable(tf.random_uniform([num_feature, num_hidden]))
b1 = tf.Variable(tf.random_uniform([num_hidden]))

w2 = tf.Variable(tf.random_uniform([num_hidden, num_classes]))
b2 = tf.Variable(tf.random_uniform([num_classes]))

In [17]:
#layer 1
z1 = tf.matmul(X, w1) + b1
a1 = tf.nn.tanh(z1)

#output layer
z2 = tf.matmul(a1, w2) + b2
Yhat = tf.nn.tanh(z2)

In [18]:
loss_function = tf.reduce_mean(tf.square(Yhat - Y))

In [19]:
optimizer = tf.train.AdamOptimizer(1e-2).minimize(loss_function)

In [20]:
init = tf.global_variables_initializer()

### Reptile

In [21]:
#number of epochs i.e training iterations
num_epochs = 100


#number of samples i.e number of shots
num_samples = 50  

#number of tasks
num_tasks = 2

#number of times we want to perform optimization
num_iterations = 10


#mini btach size
mini_batch = 10

In [23]:
#start the tensorflow session
with tf.Session() as sess:
    
    sess.run(init)
    
    for e in range(num_epochs):
        
        #for each task in batch of tasks
        for task in range(num_tasks):

            #get the initial parameters of the model
            old_w1, old_b1, old_w2, old_b2 = sess.run([w1, b1, w2, b2,])

            #sample x and y
            x_sample, y_sample = sample_points(num_samples)


            #for some k number of iterations perform optimization on the task
            for k in range(num_iterations):

                #get the minibatch x and y
                for i in range(0, num_samples, mini_batch):

                    #sample mini batch of examples 
                    x_minibatch = x_sample[i:i+mini_batch]
                    y_minibatch = y_sample[i:i+mini_batch]


                    train = sess.run(optimizer, feed_dict={X: x_minibatch.reshape(mini_batch,1), 
                                                           Y: y_minibatch.reshape(mini_batch,1)})

            #get the updated model parameters after several iterations of optimization
            new_w1, new_b1, new_w2, new_b2 = sess.run([w1, b1, w2, b2])

            #Now we perform meta update

            #i.e theta = theta + epsilon * (theta_star - theta)

            epsilon = 0.1

            updated_w1 = old_w1 + epsilon * (new_w1 - old_w1) 
            updated_b1 = old_b1 + epsilon * (new_b1 - old_b1) 

            updated_w2 = old_w2 + epsilon * (new_w2 - old_w2) 
            updated_b2 = old_b2 + epsilon * (new_b2 - old_b2) 


            #update the model parameter with new parameters
            w1.load(updated_w1, sess)
            b1.load(updated_b1, sess)

            w2.load(updated_w2, sess)
            b2.load(updated_b2, sess)

        if e%10 == 0:
            loss = sess.run(loss_function, feed_dict={X: x_sample.reshape(num_samples,1), Y: y_sample.reshape(num_samples,1)})

            print("Epoch {}: Loss {}\n".format(e,loss));             
            print('Updated Model Parameter Theta\n');
            print('Sampling Next Batch of Tasks \n')
            print('---------------------------------\n')

Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
Epoch 0: Loss 1.0416618585586548

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 10: Loss 0.8659334778785706

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 20: Loss 12.654692649841309

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 30: Loss 0.9814862012863159

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 40: Loss 11.113720893859863

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 50: Loss 4.278852939605713

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 60: Loss 0.8769494891166687

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

-------------------------